In [ ]:
!pip uninstall -y flair
!pip install flair==0.10

In [ ]:
from flair.datasets import ColumnCorpus
from flair.embeddings import WordEmbeddings
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.visual.training_curves import Plotter

In [ ]:
data_folder = "."
columns = {0: 'text', 1: 'ner'}

In [ ]:
corpus = ColumnCorpus(data_folder, columns,
                              train_file='/kaggle/input/legalner/preamble_train.txt',
                            dev_file='/kaggle/input/legalner/preamble_dev.txt')

In [ ]:
 tag_dictionary = corpus.make_label_dictionary('ner')

# BERT CRF

In [ ]:
embeddings = TransformerWordEmbeddings('dslim/bert-base-NER', fine_tune=True, layers='-1') # 'Jean-Baptiste/camembert-ner') # WordEmbeddings('glove')

In [ ]:
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=tag_dictionary,
                        tag_type='ner',
                        use_crf=True,
                       use_rnn=False,
                       reproject_embeddings=False)

In [ ]:
trainer = ModelTrainer(tagger, corpus)

In [ ]:
trainer.train('bert-crf',
              learning_rate=0.05,
              mini_batch_size=16,
              max_epochs=100)

In [ ]:
!tar -cvf bert-crf-fine-tune.tar ./bert-crf
!gzip bert-crf-fine-tune.tar

In [ ]:
plotter = Plotter()
plotter.plot_training_curves('loss.tsv')

# BERT-BiLSTM-CRF

In [ ]:
embeddings2 = TransformerWordEmbeddings('dslim/bert-base-NER', fine_tune=True, layers='-1') # 'Jean-Baptiste/camembert-ner') # WordEmbeddings('glove')

In [ ]:
tagger2 = SequenceTagger(hidden_size=256,
                        embeddings=embeddings2,
                        tag_dictionary=tag_dictionary,
                        tag_type='ner',
                        use_crf=True,
                       reproject_embeddings=False)

In [ ]:
trainer2 = ModelTrainer(tagger2, corpus)

In [ ]:
trainer2.train('bert-bilstm-crf',
              learning_rate=0.05,
              mini_batch_size=4,
              max_epochs=100)

In [ ]:
!tar -cvf bert-bilstm-crf.tar ./bert-bilstm-crf
!gzip bert-bilstm-crf.tar

In [ ]:
plotter = Plotter()
plotter.plot_training_curves('bert-bilstm-crf/loss.tsv')

# GLoVe + Char + Flair + BiLSTM + CRF

In [ ]:
from flair.embeddings import WordEmbeddings, StackedEmbeddings, FlairEmbeddings

In [ ]:
embedding_types = [

    WordEmbeddings('glove'),

    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]

embeddings3 = StackedEmbeddings(embeddings=embedding_types)

In [ ]:
tagger3 = SequenceTagger(hidden_size=256,
                        embeddings=embeddings3,
                        tag_dictionary=tag_dictionary,
                        tag_type='ner',
                        use_crf=True,
                       reproject_embeddings=False)

In [ ]:
trainer3 = ModelTrainer(tagger3, corpus)

In [ ]:
trainer3.train('bilstm-crf-glove-flair',
              learning_rate=0.05,
              mini_batch_size=32,
              max_epochs=100)

In [ ]:
!tar -cvf bilstm-crf-glove-flair.tar bilstm-crf-glove-flair
!gzip bilstm-crf-glove-flair.tar

In [ ]:
plotter = Plotter()
plotter.plot_training_curves('bilstm-crf-glove-flair/loss.tsv')

# BERT + linear

In [ ]:
embeddings = TransformerWordEmbeddings('dslim/bert-base-NER') # 'Jean-Baptiste/camembert-ner') # WordEmbeddings('glove')

In [ ]:
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=tag_dictionary,
                        tag_type='ner',
                        use_crf=False,
                       use_rnn=False,
                       reproject_embeddings=True)

In [ ]:
trainer = ModelTrainer(tagger, corpus)

In [ ]:
!rm -rf flair-model

In [ ]:
trainer.train('bert-linear',
              learning_rate=0.01,
              mini_batch_size=32,
              max_epochs=100)

In [ ]:
!tar -cvf bert-linear.tar ./bert-linear
!gzip bert-linear.tar

In [ ]:
plotter = Plotter()
plotter.plot_training_curves('bert-linear/loss.tsv')